In [1]:
import numpy as np
import time
import os
from tqdm import tqdm
import pyarrow.parquet as pq
from sklearn.neighbors import KDTree
import json
import random

DATA_FILE = "ad_features_all_gt1_14_08.parquet"
FILE_WITHOUT_MEDIAN = "kurtosis_r_without_median.json"
FILE_WITH_MEDIAN = "kurtosis_r_with_median.json"
REPORT_FILE = "comparison_report.txt"

NUM_NEIGHBORS = 10
NUM_RANDOM_TARGETS = 500
TARGET_FEATURE = 'kurtosis_r'

FEATURE_NAMES = [
    'mean_g', 'weighted_mean_g',
    'standard_deviation_g', 'median_g', 'amplitude_g', 'beyond_1_std_g',
    'cusum_g', 'inter_percentile_range_10_g', 'kurtosis_g', 'linear_trend_g',
    'linear_trend_sigma_g', 'linear_trend_noise_g', 'linear_fit_slope_g',
    'linear_fit_slope_sigma_g', 'linear_fit_reduced_chi2_g',
    'magnitude_percentage_ratio_40_5_g', 'magnitude_percentage_ratio_20_10_g',
    'maximum_slope_g', 'median_absolute_deviation_g',
    'median_buffer_range_percentage_10_g', 'percent_amplitude_g',
    'anderson_darling_normal_g', 'chi2_g', 'skew_g', 'stetson_K_g', 'mean_r',
    'weighted_mean_r', 'standard_deviation_r', 'median_r', 'amplitude_r',
    'beyond_1_std_r', 'cusum_r', 'inter_percentile_range_10_r', 'kurtosis_r',
    'linear_trend_r', 'linear_trend_sigma_r', 'linear_trend_noise_r',
    'linear_fit_slope_r', 'linear_fit_slope_sigma_r', 'linear_fit_reduced_chi2_r',
    'magnitude_percentage_ratio_40_5_r', 'magnitude_percentage_ratio_20_10_r',
    'maximum_slope_r', 'median_absolute_deviation_r',
    'median_buffer_range_percentage_10_r', 'percent_amplitude_r',
    'anderson_darling_normal_r', 'chi2_r', 'skew_r', 'stetson_K_r', 'distnr'
]

def load_raw_data(filepath):
    """Loads raw data from Parquet without dropping NaNs immediately."""
    print(f"Reading data from '{filepath}'...")
    try:
        pq_file = pq.ParquetFile(filepath)
    except Exception as e:
        print(f" ERROR: Could not open Parquet file '{filepath}'. {e}"); return None, None

    object_ids_list, feature_batches = [], []
    columns_to_read = ['objectId'] + FEATURE_NAMES

    for i in tqdm(range(pq_file.num_row_groups), desc="Reading batches"):
        batch = pq_file.read_row_group(i, columns=columns_to_read)
        object_ids_list.extend(batch.column('objectId').to_pylist())
        feature_batches.append(np.column_stack([batch.column(name).to_numpy() for name in FEATURE_NAMES]))

    data_matrix = np.vstack(feature_batches).astype('float32')
    object_ids = np.array(object_ids_list)
    print(f"Raw data loaded. Shape: {data_matrix.shape}")
    return object_ids, data_matrix

def process_and_search(raw_ids, raw_data, target_ids, mode='drop_nan'):
    """
    Processes data (normalization, cleaning/imputing), builds KDTree, and searches.
    mode: 'drop_nan' (standard) or 'fill_median' (impute kurtosis_r).
    """
    print(f"\n--- Processing Mode: {mode} ---")
    

    data_processed = raw_data.copy()
    
    if mode == 'fill_median':
        if TARGET_FEATURE in FEATURE_NAMES:
            idx = FEATURE_NAMES.index(TARGET_FEATURE)

            median_val = np.nanmedian(data_processed[:, idx])
            print(f"   Filling NaN in '{TARGET_FEATURE}' with median: {median_val:.4f}")

            col_data = data_processed[:, idx]
            mask_nan = np.isnan(col_data)
            data_processed[mask_nan, idx] = median_val
            print(f"   Filled {np.sum(mask_nan)} values.")
        else:
            print(f"   Warning: Feature {TARGET_FEATURE} not found.")

    nan_mask = np.isnan(data_processed).any(axis=1)
    clean_data = data_processed[~nan_mask]
    clean_ids = raw_ids[~nan_mask]
    print(f"   Rows after cleaning NaNs: {clean_data.shape[0]:,}")


    print("   Normalizing...")
    mean = np.mean(clean_data, axis=0, dtype='float32')
    std = np.std(clean_data, axis=0, dtype='float32')
    std[std == 0] = 1.0
    norm_data = (clean_data - mean) / std


    print("   Building KD-Tree...")
    start_time = time.time()
    kdt = KDTree(norm_data, leaf_size=40, metric='euclidean')
    print(f"   Tree built in {time.time() - start_time:.2f} sec.")

    # 5. Prepare Query Vectors
    # Map IDs to new indices in the cleaned/normalized array
    id_to_idx = {oid: i for i, oid in enumerate(clean_ids)}
    
    query_vectors = []
    valid_target_ids = []
    
    for tid in target_ids:
        if tid in id_to_idx:
            query_vectors.append(norm_data[id_to_idx[tid]])
            valid_target_ids.append(tid)
        else:
            # Should not happen if we selected targets from clean rows initially, 
            # but good for safety.
            pass
            
    query_vectors = np.array(query_vectors, dtype='float32')
    
    # 6. Search
    print(f"   Searching neighbors for {len(query_vectors)} targets...")
    # k + 1 because the point itself is included
    distances, indices = kdt.query(query_vectors, k=NUM_NEIGHBORS + 1)

    # 7. Format Results
    results = {}
    for i, tid in enumerate(valid_target_ids):
        neighbor_idxs = indices[i]
        # neighbors_list = []
        neighbor_ids_only = []
        
        count = 0
        for n_idx in neighbor_idxs:
            n_id = clean_ids[n_idx]
            if n_id == tid:
                continue # Skip self
            if count >= NUM_NEIGHBORS:
                break
            neighbor_ids_only.append(n_id)
            count += 1
            
        results[tid] = neighbor_ids_only
        
    return results

def select_random_targets(ids, data, n=500):
    """Selects N random objects that define the 'Gold Standard' (must not have NaNs)."""
    print(f"\nSelecting {n} random target objects (skipping rows with NaN)...")

    valid_mask = ~np.isnan(data).any(axis=1)
    valid_ids = ids[valid_mask]
    
    if len(valid_ids) < n:
        print(f"Warning: Only {len(valid_ids)} valid objects found. Using all.")
        selected = valid_ids
    else:

        selected = np.random.choice(valid_ids, n, replace=False)
        
    print(f"Selected {len(selected)} targets.")
    return selected.tolist()

def compare_and_report(res1, res2, output_txt):
    print(f"\nComparing results and writing report to {output_txt}...")
    
    common_targets = set(res1.keys()).intersection(set(res2.keys()))
    total_targets = len(common_targets)
    
    if total_targets == 0:
        print("Error: No common targets found between runs.")
        return

    total_overlap_percent = 0.0
    
    with open(output_txt, 'w', encoding='utf-8') as f:
        f.write("NEAREST NEIGHBOR COMPARISON REPORT\n")
        f.write("==================================\n")
        f.write(f"Total targets compared: {total_targets}\n")
        f.write(f"Neighbors per target: {NUM_NEIGHBORS}\n")
        f.write(f"Comparison: Drop NaN vs. Fill NaN with Median ({TARGET_FEATURE})\n\n")
        f.write(f"{'ObjectID':<20} | {'Overlap (Cnt)':<15} | {'Overlap (%)':<15}\n")
        f.write("-" * 60 + "\n")
        
        for tid in common_targets:
            set1 = set(res1[tid])
            set2 = set(res2[tid])
            
            intersection = set1.intersection(set2)
            overlap_count = len(intersection)
            overlap_pct = (overlap_count / NUM_NEIGHBORS) * 100
            
            total_overlap_percent += overlap_pct
            
            f.write(f"{tid:<20} | {overlap_count:<15} | {overlap_pct:<15.1f}\n")
        
        avg_overlap = total_overlap_percent / total_targets
        
        f.write("-" * 60 + "\n")
        f.write(f"AVERAGE OVERLAP PERCENTAGE: {avg_overlap:.2f}%\n")
        
    print(f"Report generated. Average overlap: {avg_overlap:.2f}%")

# --- Main execution block ---
if __name__ == "__main__":
    if not os.path.exists(DATA_FILE):
        print(f"ERROR: Data file '{DATA_FILE}' not found.")
        exit()

    # 1. Load Raw Data
    all_ids, raw_data = load_raw_data(DATA_FILE)
    if all_ids is None:
        exit()

    # 2. Select 500 random targets (must be valid to be fair comparison)
    target_objects = select_random_targets(all_ids, raw_data, NUM_RANDOM_TARGETS)

    # 3. Run 1: Standard (Drop NaNs) -> 'without_median' (implies no imputation)
    results_without = process_and_search(all_ids, raw_data, target_objects, mode='drop_nan')
    
    # 4. Save Run 1
    with open(FILE_WITHOUT_MEDIAN, 'w', encoding='utf-8') as f:
        json.dump(results_without, f, indent=4)
    print(f"Saved results to {FILE_WITHOUT_MEDIAN}")

    # 5 & 6 & 7. Run 2: Fill kurtosis_r with Median -> 'with_median'
    results_with = process_and_search(all_ids, raw_data, target_objects, mode='fill_median')

    # 8. Save Run 2
    with open(FILE_WITH_MEDIAN, 'w', encoding='utf-8') as f:
        json.dump(results_with, f, indent=4)
    print(f"Saved results to {FILE_WITH_MEDIAN}")

    # 9. Compare
    compare_and_report(results_without, results_with, REPORT_FILE)

Reading data from 'ad_features_all_gt1_14_08.parquet'...


Reading batches: 100%|████████████████████████████| 1/1 [00:07<00:00,  7.80s/it]


Raw data loaded. Shape: (3922520, 51)

Selecting 500 random target objects (skipping rows with NaN)...
Selected 500 targets.

--- Processing Mode: drop_nan ---
   Rows after cleaning NaNs: 279,763
   Normalizing...
   Building KD-Tree...
   Tree built in 1.36 sec.
   Searching neighbors for 500 targets...
Saved results to kurtosis_r_without_median.json

--- Processing Mode: fill_median ---
   Filling NaN in 'kurtosis_r' with median: -0.1591
   Filled 3221619 values.
   Rows after cleaning NaNs: 279,763
   Normalizing...
   Building KD-Tree...
   Tree built in 1.28 sec.
   Searching neighbors for 500 targets...
Saved results to kurtosis_r_with_median.json

Comparing results and writing report to comparison_report.txt...
Report generated. Average overlap: 100.00%
